In [130]:
import re
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, auc, accuracy_score, roc_auc_score, f1_score, classification_report, roc_curve, log_loss

RAND = 42

In [131]:
data = pd.read_csv('Data/SMS Spam Collection Dataset/spam.csv', delimiter=',', encoding = "ISO-8859-1")
data = pd.DataFrame(data[['v1', 'v2']])
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [132]:
data.rename(columns = {'v1' : 'spam', 'v2' : 'text'}, inplace = True)
data['spam'] = data['spam'].apply(lambda x: 0 if x == 'ham' else 1)
data.head()

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [133]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RAND)
for train_index, test_index in split.split(data, data['spam']):
    df = data.loc[train_index]
    test = data.loc[test_index]

In [134]:
def tokenize(message):
    message = message.lower()
    all_words = re.findall("[a-z0-9]+", message)
    return set(all_words)

In [135]:
def count_words(data):
    counts = defaultdict(lambda: [0,0])
    for index, row in data.iterrows():
        for word in tokenize(row['text']):
            counts[word][0 if row['spam'] else 1] +=1
    return counts

In [136]:
freq = count_words(df)
freq

defaultdict(<function __main__.count_words.<locals>.<lambda>()>,
            {'on': [89, 300],
             'going': [3, 135],
             'great': [11, 76],
             'nothing': [3, 28],
             'bye': [0, 5],
             'wat': [1, 89],
             'so': [18, 336],
             'the': [133, 687],
             'i': [33, 1632],
             's': [63, 338],
             'wont': [0, 28],
             'wit': [0, 12],
             'guys': [3, 21],
             'siva': [0, 2],
             'askd': [0, 8],
             'sry': [0, 3],
             'k': [1, 90],
             'tats': [0, 1],
             'knw': [0, 12],
             'y': [0, 39],
             '2': [121, 205],
             'ok': [4, 215],
             'can': [23, 315],
             'heart': [3, 31],
             'be': [46, 242],
             'you': [194, 1090],
             'your': [181, 294],
             'stand': [0, 7],
             'of': [73, 362],
             'are': [58, 288],
             'crave': [0, 9],
     

In [137]:
all_spam = df['spam'].loc[df['spam'] == 1].count()
all_not_spam = df['spam'].loc[df['spam'] == 0].count()

In [138]:
def prob(freq, all_spam, all_not_spam, k = 1):
    return [ (word, 
            (frequency[0] + k) / (all_spam + 2*k),
            (frequency[1] + k) / (all_not_spam + 2*k))
            for word, frequency in counts.items()]

In [139]:
result = prob(freq, all_spam, all_not_spam)
result

[('world', 0.0033333333333333335, 0.009065009065009065),
 ('great', 0.02, 0.025382025382025383),
 ('there', 0.028333333333333332, 0.05128205128205128),
 ('crazy', 0.008333333333333333, 0.002849002849002849),
 ('n', 0.02, 0.030044030044030043),
 ('available', 0.006666666666666667, 0.004144004144004144),
 ('la', 0.0016666666666666668, 0.002072002072002072),
 ('only', 0.14166666666666666, 0.03289303289303289),
 ('amore', 0.0016666666666666668, 0.000518000518000518),
 ('go', 0.05333333333333334, 0.060088060088060086),
 ('e', 0.018333333333333333, 0.019943019943019943),
 ('got', 0.013333333333333334, 0.057498057498057496),
 ('cine', 0.0016666666666666668, 0.002072002072002072),
 ('jurong', 0.0016666666666666668, 0.000518000518000518),
 ('bugis', 0.0016666666666666668, 0.002072002072002072),
 ('wat', 0.0033333333333333335, 0.02641802641802642),
 ('point', 0.0016666666666666668, 0.003626003626003626),
 ('in', 0.11833333333333333, 0.19062419062419061),
 ('until', 0.01, 0.005957005957005957),
 

In [198]:
import math
def final(word_probs, message):
    message_words = tokenize(message)
    spam_prob = not_spam_prob = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in message_words:
            spam_prob += math.log(prob_if_spam)
            not_spam_prob += math.log(prob_if_not_spam)
        else:
            spam_prob += math.log(1.0 - prob_if_spam)
            not_spam_prob += math.log(1.0 - prob_if_not_spam)
    e_spam_prob = math.exp(spam_prob)
    e_not_spam_prob = math.exp(not_spam_prob)
    return e_spam_prob/(e_spam_prob+e_not_spam_prob)

In [199]:
df_test = pd.DataFrame()
for index, row in test.iterrows():
    df_test = df_test.append({'text': row['text'],'prob': final(result, row['text']), 'spam_': row['spam']}, ignore_index=True)

In [181]:
df_test

,text,prob,spam_,spam_final
0,"Oh right, ok. I'll make sure that i do loads o...",3.080121e-18,0.0,0
1,I am in tirupur. call you da.,7.441478e-14,0.0,0
2,No that just means you have a fat head,1.475234e-12,0.0,0
3,"You have won ?1,000 cash or a ?2,000 prize! To...",9.999941e-01,1.0,1
4,Come aftr &lt;DECIMAL&gt; ..now i m cleaning ...,2.897699e-18,0.0,0
...,...,...,...,...
1110,Stop the story. I've told him i've returned it...,6.657080e-16,0.0,0
1111,We have sent JD for Customer Service cum Accou...,2.144272e-03,0.0,0
1112,Just re read it and I have no shame but tell m...,3.068339e-17,0.0,0
1113,"Well, I meant as opposed to my drunken night o...",1.099828e-13,0.0,0


In [232]:
def func(row):
    if row['prob'] > 0.5:
        return 1
    else:
        return 0

In [233]:
df_test['spam_final'] = df_test.apply(func, axis = 1)

In [234]:
precision_score(df_test['spam_'], df_test['spam_final'])

1.0

In [235]:
recall_score(df_test['spam_'], df_test['spam_final'])

0.9060402684563759

In [236]:
roc_auc_score(df_test['spam_'], df_test['spam_final'])

0.9530201342281879